In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
folder_path = '/content/drive/My Drive/eeg data'
file_list = os.listdir(folder_path)
files = [f for f in file_list if f.endswith('.csv')]
final_df=[]


In [9]:
for file in files:
  file_path = os.path.join(folder_path, file)
  df = pd.read_csv(file_path)
  df.columns =['Output1', 'Output2', 'Output3', 'Output4', 'var1', 'var2', 'var3', 'var4']
  df = df.iloc[:, :4]
  st_marks = ['start zero.png', 'start one.png', 'start two.jpg', 'start three.jpg', 'start four.jpg', 'start five.jpg', 'start six.jpg', 'start seven.jpg', 'start eight.jpg','start nine.jpg','start animal.jpg']
  end_marks=['end zero.png', 'end one.png', 'end two.jpg', 'end three.jpg', 'end four.jpg', 'end five.jpg', 'end six.jpg', 'end seven.jpg', 'end eight.jpg','end nine.jpg','end animal.jpg']
  df1=[]
  merged_df = pd.DataFrame()
  numerical_labels=st_marks[:-1]
  nonnum_label=st_marks[-1]



  for start,end in zip(st_marks, end_marks):
    start_idx = df[df['Output1'] ==start].index
    end_idx = df[df['Output1'] ==end].index
    temp_df=[]

    # Slice the DataFrame from start to end index
    for i,j in zip(start_idx, end_idx):
        row = df[i+1:j]
        row['numerical'] = 1 if start in numerical_labels else 0
        temp_df.append(row)
    merged_df=pd.concat(temp_df,ignore_index=True,sort=False)
    merged_df = merged_df.reset_index(drop=True)
    final_df.append(merged_df)

print(final_df)



<ipython-input-9-6fa1574ba56a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['numerical'] = 1 if start in numerical_labels else 0
<ipython-input-9-6fa1574ba56a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['numerical'] = 1 if start in numerical_labels else 0
<ipython-input-9-6fa1574ba56a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

[     Output1 Output2 Output3 Output4  numerical
0     197158   10561  806458  703133          1
1     196998   10522  806103  702832          1
2     196741   10531  805028  701851          1
3     196962   10609  805544  702280          1
4     197226   10629  806467  703101          1
...      ...     ...     ...     ...        ...
1275  199158   15200  816869  712238          1
1276  199255   15258  816928  712270          1
1277  199304   15263  816946  712274          1
1278  199230   15210  816909  712257          1
1279  199146   15183  816879  712249          1

[1280 rows x 5 columns],      Output1 Output2 Output3 Output4  numerical
0     193746    3107  789263  688324          1
1     193809    3148  789205  688231          1
2     193709    3106  788867  687926          1
3     193659    3048  788988  688076          1
4     193679    3053  789231  688316          1
...      ...     ...     ...     ...        ...
1275  196056    8018  800756  698248          1
1276  195968 

In [6]:
final_combined_df = pd.concat(final_df, ignore_index=True, sort=False)

print(final_combined_df)

       Output1 Output2 Output3 Output4  numerical
0       197158   10561  806458  703133          1
1       196998   10522  806103  702832          1
2       196741   10531  805028  701851          1
3       196962   10609  805544  702280          1
4       197226   10629  806467  703101          1
...        ...     ...     ...     ...        ...
250793  325782  249311  797428  707219          0
250794  325839  249354  797467  707268          0
250795  326102  249544  797623  707372          0
250796  326178  249604  797673  707388          0
250797  326025  249493  797593  707326          0

[250798 rows x 5 columns]


In [8]:
numerical_data=final_combined_df[final_combined_df['numerical']==1]
nonnum_data=final_combined_df[final_combined_df['numerical']==0]
from sklearn.utils import resample
nonnum_upsampled = resample(nonnum_data,
                                   replace=True,
                                   n_samples=len(numerical_data),
                                   random_state=42)
new_data=pd.concat([numerical_data,nonnum_upsampled])
new_data=new_data.sample(frac=1,random_state=42).reset_index(drop=True)
print(new_data)

       Output1 Output2  Output3  Output4  numerical
0       296055  418068  2071707  2032176          0
1       204424   53570   430976   333958          1
2       471974  251606   827053   639794          1
3       200584   18067   823164   717666          0
4       482533  364449  1177463  1010651          0
...        ...     ...      ...      ...        ...
456137  200466   17898   822861   717436          0
456138  221711   63338   479240   369039          0
456139  313129  120577    86722    58069          1
456140  377351  116439    91723    48833          1
456141  286486  141136    96962    78726          1

[456142 rows x 5 columns]


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
sc=StandardScaler()
X=new_data.drop(columns=['numerical']).values
y=new_data['numerical'].values
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(364913, 4, 1)
(91229, 4, 1)
(364913,)
(91229,)


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D,Flatten, Dropout

model = Sequential([
    Conv1D(filters=32, kernel_size=4, activation='relu', input_shape=(4, 1)),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
11404/11404 [==============================] - 34s 3ms/step - loss: 0.5401 - accuracy: 0.7044 - val_loss: 0.3293 - val_accuracy: 0.8669
Epoch 2/20
11404/11404 [==============================] - 30s 3ms/step - loss: 0.3281 - accuracy: 0.8432 - val_loss: 0.2469 - val_accuracy: 0.8766
Epoch 3/20
11404/11404 [==============================] - 31s 3ms/step - loss: 0.2807 - accuracy: 0.8632 - val_loss: 0.2114 - val_accuracy: 0.8980
Epoch 4/20
11404/11404 [==============================] - 29s 3ms/step - loss: 0.2598 - accuracy: 0.8718 - val_loss: 0.1934 - val_accuracy: 0.9055
Epoch 5/20
11404/11404 [==============================] - 29s 3ms/step - loss: 0.2474 - accuracy: 0.8758 - val_loss: 0.1955 - val_accuracy: 0.9048
Epoch 6/20
11404/11404 [==============================] - 29s 3ms/step - loss: 0.2442 - accuracy: 0.8770 - val_loss: 0.1888 - val_accuracy: 0.9105
Epoch 7/20
11404/11404 [==============================] - 29s 3ms/step - loss: 0.2354 - accuracy: 0.8806 - val_loss: 0

In [16]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

2851/2851 [==============================] - 4s 2ms/step - loss: 0.1680 - accuracy: 0.9087
Test Loss: 0.16796621680259705
Test Accuracy: 0.9087132215499878


In [17]:
from sklearn.metrics import confusion_matrix,precision_score,f1_score
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
print("Precision:", precision)
print("F1 Score:", f1)

2851/2851 [==============================] - 5s 2ms/step
Confusion Matrix:
[[42432  3261]
 [ 5067 40469]]
Precision: 0.9254287674365425
F1 Score: 0.9067058006407815
